# GPT-5 新機能テスト

以下の機能をテストします。
- https://cookbook.openai.com/examples/gpt-5/gpt-5_new_params_and_tools
- https://cookbook.openai.com/examples/gpt-5/gpt-5_prompting_guide

In [ ]:
!pip install openai --upgrade

In [ ]:
from openai import OpenAI,AzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# Last generation API
# client = AzureOpenAI(
#   azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
#   api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
#   api_version="2025-04-01-preview"
# )

# Next generation API 
client = OpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=f"{os.getenv('AZURE_OPENAI_ENDPOINT')}/openai/v1/",
    default_query={"api-version": "preview"}, 
)


## 1. Verbosity Parameter（冗長性パラメーター）
モデルに返答の拡張度を高くしたり低くしたりするようヒントを与えます。コーディングタスクの場合、詳細度パラメータは、生成されるコードの長さと複雑さ、そして付随する説明の深さにも影響します。

低→ 簡潔な UX、最小限の文章。
中 (デフォルト) → バランスの取れた詳細。
高→ 詳細、監査、教育、または引き継ぎに最適。


## 素数判定（verbosity:low）

In [ ]:
response = client.responses.create(
    model="gpt-5-mini",
    input="素数を判定するPythonプログラムを出力してください。",
    text={"verbosity": "low"},
)

print(response.model_dump_json(indent=2))


In [ ]:
output_text = ""
for item in response.output:
    if hasattr(item, "content") and item.content:
        for content in item.content:
            if hasattr(content, "text") and content.text:
                output_text += content.text


print(output_text)        

In [ ]:
response.usage

## 素数判定（verbosity:medium）

In [ ]:
response = client.responses.create(
    model="gpt-5-mini", 
    input="素数を判定するPythonプログラムを出力してください。",
    text={"verbosity": "medium"},
)

print(response.model_dump_json(indent=2))


In [ ]:
output_text = ""
for item in response.output:
    if hasattr(item, "content") and item.content:
        for content in item.content:
            if hasattr(content, "text") and content.text:
                output_text += content.text

print(output_text)

## 素数判定（verbosity:high）

In [ ]:
response = client.responses.create(
    model="gpt-5-mini",
    input="素数を判定するPythonプログラムを出力してください。",
    text={"verbosity": "high"},
)

print(response.model_dump_json(indent=2))


In [ ]:
output_text = ""
for item in response.output:
    if hasattr(item, "content") and item.content:
        for content in item.content:
            if hasattr(content, "text") and content.text:
                output_text += content.text

print(output_text)

## 2. Free-Form Function Calling（自由形式の関数呼び出し）
Python スクリプトから SQL クエリまで、あらゆる生のテキストペイロードを JSON ラップなしで直接カスタムツールに連携します。


In [ ]:
response = client.responses.create(
    model="gpt-5-mini",
    input="code_exec ツールを使用して、strawberry の 'r' の文字数に等しい半径を持つ円の面積を計算してください。",
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "code_exec",
            "description": "Executes arbitrary python code",
        }
    ]
)
print(response.output)


In [ ]:
# response.outputからinputの中身を取得
for item in response.output:
    if hasattr(item, 'type') and item.type == 'custom_tool_call':
        print("=== Generated Code ===")
        print(item.input)
        print("======================")


## 3. Allowed tools
`allowed_tools` パラメーターは、GPT-5 Responses API の `tool_choice` 内で、N 個のツール定義を指定しつつ、モデルが使用できるツールを M（N 未満）に制限します。ツールの完全なリストを `tools` に列挙し、`allowed_tools` ブロックを使用してサブセットを指定し、モードを指定します。モードは `auto` または `required` のいずれかが指定可能。

https://platform.openai.com/docs/guides/latest-model#allowed-tools



In [ ]:
# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "指定された都市の現在の天気を取得します。",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "都市名（例：Tokyo）",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["location"],
        },
    },
    {
        "type": "function",
        "name": "search_products",
        "description": "指定されたキーワードに基づいて商品を検索します。",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "検索したい商品名やキーワード（例：ノートパソコン、トースター）",
                },
            },
            "required": ["query"],
        },
    }
]

prompt = """
あなたは便利なアシスタントです。質問に答えるために、必要に応じてツールを使用します。
"""


messages = [
    {"role": "developer", "content": prompt },
    {"role": "user", "content": "草津温泉の現在の天気は？"},
]

response = client.responses.create(
    model="gpt-5-mini",
    input=messages,
    tools=tools,
    # tool_choice="auto",
    tool_choice={
        "type": "allowed_tools",
        "mode": "auto",
        "tools": [
            {"type": "function", "name": "get_current_weather"},
            {"type": "function", "name": "search_products"}
        ]
    }
)

In [ ]:
print("Final output:")
print(response.model_dump_json(indent=2))

In [ ]:
response.output[1]

## 4. Context-Free Grammar (CFG)

CFG は、文法規則を用いて文字列を生成するための形式的なルールセットであり、これらの規則は「生成規則」として定義され、非終端記号と終端記号から構成されます。CFG はパーサーによって入力文字列が文法に従っているかどうかを判断するために使われます。これは、プログラミング言語の構文や OpenAI ツール用のカスタムフォーマットに出力を制約する際に役立ちます。

この仕組みを利用することで、モデルが文法に従った有効な文字列のみを出力することを保証できます。モデルのサンプリングを制限するために、内部で LLGuidance を使用しているとの記述がありました。

#### サポートされている文法構文
- Lark - https://lark-parser.readthedocs.io/en/stable/
- 正規表現 - https://docs.rs/regex/latest/regex/#syntax

### 1. Lark 文法 CFG 構文
OpenAI の例に従って、SQLの 特定の構文パターン を解析・生成するためのルールセットを Lark 文法で定義します。"SELECT ... FROM ... WHERE ... ORDER BY ..." のような典型的なクエリパターンのみをサポートしたい場合は以下のように記述します。


In [ ]:
import textwrap

# ----------------- grammars for MS SQL dialect -----------------
mssql_grammar = textwrap.dedent(r"""
            // ---------- Punctuation & operators ----------
            SP: " "
            COMMA: ","
            GT: ">"
            EQ: "="
            SEMI: ";"

            // ---------- Start ----------
            start: "SELECT" SP "TOP" SP NUMBER SP select_list SP "FROM" SP table SP "WHERE" SP amount_filter SP "AND" SP date_filter SP "ORDER" SP "BY" SP sort_cols SEMI

            // ---------- Projections ----------
            select_list: column (COMMA SP column)*
            column: IDENTIFIER

            // ---------- Tables ----------
            table: IDENTIFIER

            // ---------- Filters ----------
            amount_filter: "total_amount" SP GT SP NUMBER
            date_filter: "order_date" SP GT SP DATE

            // ---------- Sorting ----------
            sort_cols: "order_date" SP "DESC"

            // ---------- Terminals ----------
            IDENTIFIER: /[A-Za-z_][A-Za-z0-9_]*/
            NUMBER: /[0-9]+/
            DATE: /'[0-9]{4}-[0-9]{2}-[0-9]{2}'/
    """)

# ----------------- grammars for PostgreSQL dialect -----------------
postgres_grammar = textwrap.dedent(r"""
            // ---------- Punctuation & operators ----------
            SP: " "
            COMMA: ","
            GT: ">"
            EQ: "="
            SEMI: ";"

            // ---------- Start ----------
            start: "SELECT" SP select_list SP "FROM" SP table SP "WHERE" SP amount_filter SP "AND" SP date_filter SP "ORDER" SP "BY" SP sort_cols SP "LIMIT" SP NUMBER SEMI

            // ---------- Projections ----------
            select_list: column (COMMA SP column)*
            column: IDENTIFIER

            // ---------- Tables ----------
            table: IDENTIFIER

            // ---------- Filters ----------
            amount_filter: "total_amount" SP GT SP NUMBER
            date_filter: "order_date" SP GT SP DATE

            // ---------- Sorting ----------
            sort_cols: "order_date" SP "DESC"

            // ---------- Terminals ----------
            IDENTIFIER: /[A-Za-z_][A-Za-z0-9_]*/
            NUMBER: /[0-9]+/
            DATE: /'[0-9]{4}-[0-9]{2}-[0-9]{2}'/
    """)


### PostgreSQL 生成

In [ ]:
sql_prompt_pg = """
postgres_grammar を呼び出して、PostgreSQL 用のクエリを生成し、顧客ごとに最新の 5 件の注文を取得します。
取得するカラムは customer_id、order_id、order_date、および total_amount です。
ただし、total_amount が 500 を超え、order_date が 『2025-01-01』 以降である条件を満たすレコードのみを取得します。
"""

response_pg = client.responses.create(
    model="gpt-5-mini",
    input=sql_prompt_pg,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "postgres_grammar",
            "description": "Executes read-only PostgreSQL queries limited to SELECT statements with LIMIT and basic WHERE/ORDER BY. YOU MUST REASON HEAVILY ABOUT THE QUERY AND MAKE SURE IT OBEYS THE GRAMMAR.",
            "format": {
                "type": "grammar",
                "syntax": "lark",
                "definition": postgres_grammar
            }
        },
    ],
    parallel_tool_calls=False,
)

print("--- PG SQL Query ---")
print(response_pg.output[1].input)


### MS SQL 生成

In [ ]:
sql_prompt_pg = """
mssql_grammar を呼び出して、Microsoft SQL Server 用のクエリを生成し、顧客ごとに最新の 5 件の注文を取得します。
取得するカラムは customer_id、order_id、order_date、および total_amount です。
ただし、total_amount が 500 を超え、order_date が 『2025-01-01』 以降である条件を満たすレコードのみを取得します。
"""

response_pg = client.responses.create(
    model="gpt-5-mini",
    input=sql_prompt_pg,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "mssql_grammar",
            "description": "Executes read-only Microsoft SQL Server queries limited to SELECT statements with LIMIT and basic WHERE/ORDER BY. YOU MUST REASON HEAVILY ABOUT THE QUERY AND MAKE SURE IT OBEYS THE GRAMMAR.",
            "format": {
                "type": "grammar",
                "syntax": "lark",
                "definition": mssql_grammar
            }
        },
    ],
    parallel_tool_calls=False,
)

print("--- MS SQL Query ---")
print(response_pg.output[1].input)


### 例：許可された最小限のタグだけを含むシンプルな HTML 断片 を生成

In [ ]:
safe_html_grammar = textwrap.dedent(r"""
    start: elements
    elements: element*
    element: tag_open content tag_close | self_closing_tag
    tag_open: "<" tag_name ">"
    tag_close: "</" tag_name ">"
    self_closing_tag: "<" tag_name "/>"
    tag_name: "p" | "b" | "i" | "strong" | "em" | "br"
    content: (text | element)*
    text: /[^<]+/
""")


In [ ]:
html_prompt = (
        "safe_html_grammar を呼び出して、シンプルな僕のホームページを生成します。"
)

response_mssql = client.responses.create(
    model="gpt-5-mini",
    input=html_prompt,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "safe_html_grammar",
            "description": "Saves a safe HTML snippet.",
            "format": {
                "type": "grammar",
                "syntax": "lark",
                "definition": safe_html_grammar
            }
        },
    ],
    parallel_tool_calls=False
)

print("--- HTML ---")
print(response_mssql.output[1].input)


LLGuidance Syntax – https://github.com/guidance-ai/llguidance/blob/main/docs/syntax.md

### 制限
文法の一部はサポートされていないものもあるので注意。ベストプラクティスとトラブルシューティングを参照。

#### サポートされていない Lark 文法
- Lookaround in regexes ((?=...), (?!...), etc.)
- Lazy modifier (*?, +?, ??) in regexes.
- Terminal priorities, templates, %declares, %import (except %import common).

https://cookbook.openai.com/examples/gpt-5/gpt-5_new_params_and_tools#35-best-practices


### 2. 正規表現 CFG 構文

In [ ]:
timestamp_grammar_definition = r"^\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]) (?:[01]\d|2[0-3]):[0-5]\d$"

timestamp_prompt = (
        "timestamp_grammar を呼び出して、2025年8月7日午前10時のタイムスタンプを生成します。"
)

response_mssql = client.responses.create(
    model="gpt-5-mini",
    input=timestamp_prompt,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "timestamp_grammar",
            "description": "Saves a timestamp in date + time in 24-hr format.",
            "format": {
                "type": "grammar",
                "syntax": "regex",
                "definition": timestamp_grammar_definition
            }
        },
    ],
    parallel_tool_calls=False
)

print("--- Timestamp ---")
print(response_mssql.output[1].input)


### 制限
正規表現は Rust の正規表現ライブラリ（crate）の構文を使用し、Python の re モジュールは使用しません。正規表現の以下の機能はサポートされていません:

- Look around
- Lazy 量指定子 (*?, +?, ??)

## 5. Minimal Reasoning
GPT-5 を推論トークンをほとんどまたは全く使用せずに実行することで、レイテンシを最小限に抑え、最初のトークン生成時間を短縮します。説明を必要としない、決定論的で軽量なタスク（抽出、フォーマット、短い書き換え、単純な分類など）に最適です。指定がない場合、effort はデフォルトで medium になります。


In [ ]:
prompt = "レビューの感情を「ポジティブ」「ニュートラル」「ネガティブ」の3つに分類してください。1つの単語のみを返してください。" 

response = client.responses.create(
    model="gpt-5",
    input= [{ 'role': 'developer', 'content': prompt }, 
            { 'role': 'user', 'content': 'そのレストランの料理は本当に美味しかったです！みんなにオススメします。' }],
    reasoning = {
        "effort": "minimal"
    },
)

response.output[1].content[0].text


In [ ]:
response.usage

## 6. Preambles（序文）
Preambles は、GPT- 5がツールや関数を呼び出す前に生成する、ユーザーに見える短い説明です。その目的や計画（例：「このツールを呼び出す理由」）を説明します。Preambles は、思考の連鎖の後に表示され、実際のツール呼び出しの前に配置されます。これにより、モデルの推論プロセスが透明化し、デバッグのしやすさ、ユーザーの信頼性、および細かな制御可能性が向上します。

In [ ]:
prompt = """
<tool_preambles>
- Always begin by rephrasing the user's goal in a friendly, clear, and concise manner, before calling any tools.
- Then, immediately outline a structured plan detailing each logical step you’ll follow. - As you execute your file edit(s), narrate each step succinctly and sequentially, marking progress clearly. 
- Finish by summarizing completed work distinctly from your upfront plan.
</tool_preambles>
"""

response = client.responses.create(
    model="gpt-5-mini",
    input= [{ 'role': 'developer', 'content': prompt }, 
        { 'role': 'user', 'content': "code_exec ツールを使用して、strawberry の 'r' の文字数に等しい半径を持つ円の面積を計算してください。" }],
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "code_exec",
            "description": "Executes arbitrary python code",
        }
    ]
)
print(response.output)

In [ ]:
# レスポンスから構造化されたデータを取得
def parse_response_structure(response_output):
    result = {
        "reasoning": [],
        "messages": [],
        "tool_calls": []
    }
    
    for item in response_output:
        if hasattr(item, 'type'):
            if item.type == 'reasoning':
                result["reasoning"].append({
                    "id": item.id,
                    "summary": item.summary,
                    "status": item.status
                })
            
            elif item.type == 'message':
                message_data = {
                    "id": item.id,
                    "role": item.role,
                    "status": item.status,
                    "content": []
                }
                
                if hasattr(item, 'content') and item.content:
                    for content in item.content:
                        if hasattr(content, 'text'):
                            message_data["content"].append({
                                "type": content.type,
                                "text": content.text
                            })
                
                result["messages"].append(message_data)
            
            elif item.type == 'custom_tool_call':
                result["tool_calls"].append({
                    "id": item.id,
                    "call_id": item.call_id,
                    "name": item.name,
                    "input": item.input,
                    "status": item.status
                })
    
    return result

# 使用例
structured_data = parse_response_structure(response.output)

# きれいに表示
import json
print("=== 構造化されたレスポンス ===")
print(json.dumps(structured_data, indent=2, ensure_ascii=False))